In [64]:
# Dependencies
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from config import api_key
from datetime import datetime
api_key

'5489a88ce6a5d211411d421cb73fddea'

In [ ]:
data_df = pd.read_csv("Resources/test.csv")
#data_df.head()
full_data = pd.read_csv("Resources/train.csv")
full_data.head()

In [71]:
# Extract time and remove time zone
time = (data_df.loc[0]['pickup_datetime'])[0:-4]

dt = datetime.strptime(time, "%Y-%m-%d %H:%M:%S").timestamp()
time


'2015-01-27 13:08:24'

In [68]:
url = "http://history.openweathermap.org/data/2.5/history/city"
params = {"appid" : api_key,
              "cnt" : 1,
              "type" : "hour",
              "lat" : 40.796997,
              "lon" : -73.968124,
              "start": dt}

response = requests.get(url,params)
response.url


'http://history.openweathermap.org/data/2.5/history/city?appid=5489a88ce6a5d211411d421cb73fddea&cnt=1&type=hour&lat=40.796997&lon=-73.968124&start=1422392904.0'